In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
# GENERAL UTILITIES
import os
from glob import glob
import pandas as pd
from  tqdm.notebook import tqdm
import cv2
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd

%matplotlib inline

# MODEL DEVELOPMENT DEPENDENCIES
import numpy as np
import pywt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split


In [ ]:
def load_data_1d(gt_file_path):

  labels = pd.read_csv(gt_file_path)
  train_data, test_data = train_test_split(labels, test_size=0.2)

  return train_data, test_data

In [ ]:
gt_data_path = "/content/drive/MyDrive/BTP/train_data/train_data/train_gt.csv"

# Loading training raw data
train_data, test_data = load_data_1d(gt_data_path)

In [ ]:
train_data.shape

(1385, 5)

In [ ]:
train_data.head()

,sample_index,P,K,Mg,pH
758,758,48.5,215.0,140.0,6.3
1614,1614,97.3,222.0,151.0,6.8
836,836,74.5,243.0,137.0,6.7
287,287,43.3,220.0,198.0,6.4
596,596,46.5,180.0,179.0,7.1


In [ ]:
y_train = train_data[['K']]
y_test = test_data[['K']]
X_train = train_data.drop(columns=['sample_index', 'K'])
X_test = test_data.drop(columns=['sample_index', 'K'])

In [ ]:
X_train.head()

,P,Mg,pH
758,48.5,140.0,6.3
1614,97.3,151.0,6.8
836,74.5,137.0,6.7
287,43.3,198.0,6.4
596,46.5,179.0,7.1


In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

# Define the model
model = Sequential([
    Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(3, 1)),
    MaxPooling1D(pool_size=1),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(2)  # No activation function for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])  # Using mean squared error loss for regression

# Print model summary
model.summary()

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=8, validation_split=0.2)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 3, 64)             128       
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 3, 64)             0         
 g1D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 192)               0         
                                                                 
 dense_3 (Dense)             (None, 128)               24704     
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dense_5 (Dense)             (None, 2)                 130       
                                                      

In [ ]:
model.evaluate(X_test, y_test)

11/11 [==============================] - 0s 2ms/step - loss: 2541.3115 - mae: 37.4475


[2541.3115234375, 37.447486877441406]

In [ ]:
model.evaluate(X_test, y_test)

5/5 [==============================] - 0s 6ms/step - loss: 8.1358 - mae: 2.1782


[8.135807037353516, 2.1781575679779053]

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Print the coefficients
print("Coefficients:", model.coef_)
print("Intercept:", model.intercept_)

Mean Squared Error: 2563.4816096990457
Coefficients: [[ 0.91523416  0.45579969 27.72463699]]
Intercept: [-96.54416592]
